In [ ]:
# Note: Tensor Network does not make sense in this problem.

In [1]:
import json

import numpy as np 
import tensornetwork as tn 
import tensorflow as tf 
from tqdm import tqdm

# Data Loading

In [2]:
filepath = '/data/hok/testdata/fruit/fruits.json'

In [3]:
alldata = [json.loads(line) for line in open(filepath, 'r')]

In [4]:
alldata[0].keys()

dict_keys(['radius', 'weight', 'ripe', 'color', 'shape', 'fruit'])

In [5]:
features = {
    'quantitative': ['radius', 'weight'],
    'binary': ['ripe'],
    'qualitative': ['color', 'shape'],
    'target': 'fruit'
}

In [6]:
from collections import defaultdict

def get_feature_summary(alldata, features):
    summary = {}
    for feature in features['quantitative']:
        summary[feature] = {}
        summary[feature]['sum'] = 0.
        summary[feature]['sqsum'] = 0.
        summary[feature]['nbdata'] = 0
    for feature in features['binary']:
        summary[feature] = {}
        summary[feature][True] = 0
        summary[feature][False] = 0
    for feature in features['qualitative']:
        summary[feature] = defaultdict(lambda : 0)
        summary[features['target']] = defaultdict(lambda : 0)
        
    for datum in alldata:
        for feature in features['quantitative']:
            summary[feature]['sum'] += datum.get(feature, 0)
            summary[feature]['sqsum'] += datum.get(feature, 0)*datum.get(feature, 0)
            if datum.get(feature) is not None:
                summary[feature]['nbdata'] += 1
        for feature in features['binary']:
            if datum.get(feature) is not None and datum.get(feature) in [True, False]:
                summary[feature][datum.get(feature)] += 1
        for feature in features['qualitative']:
            val = datum.get(feature)
            if val is not None:
                summary[feature][val] += 1
        label = datum.get(features['target'])
        summary[features['target']][label] += 1
                
    for feature in features['quantitative']:
        summary[feature]['mean'] = summary[feature]['sum'] / summary[feature]['nbdata']
        summary[feature]['std'] = np.sqrt(summary[feature]['sqsum'] / summary[feature]['nbdata'] - summary[feature]['mean'] * summary[feature]['mean'])
    for feature in features['qualitative']:
        for val in summary[feature]:
            summary[feature] = dict(summary[feature])
            
    summary[features['target']] = dict(summary[features['target']])
            
    return summary

In [7]:
summary = get_feature_summary(alldata, features)

In [8]:
summary

{'radius': {'sum': 76379.23975061919,
  'sqsum': 693322.2461729473,
  'nbdata': 10000,
  'mean': 7.637923975061919,
  'std': 3.315771700293773},
 'weight': {'sum': 78789.08456281094,
  'sqsum': 828858.7404278342,
  'nbdata': 10000,
  'mean': 7.878908456281094,
  'std': 4.561652724652075},
 'ripe': {True: 1421, False: 8579},
 'color': {'green': 6195, 'red': 3805},
 'fruit': {'orange': 4008, 'durion': 2084, 'apple': 3908},
 'shape': {'circle': 2144, 'apple': 3580, 'ellipse': 4276}}

In [9]:
from scipy.sparse import dok_matrix

def transform_data_to_featurevector(alldata, features, feature_summary):
    # mapping
    feature_map = {}
    label_map = {}
    for feature in features['quantitative']:
        feature_map[feature] = len(feature_map)
    for feature in features['binary']:
        feature_map['{}:True'.format(feature)] = len(feature_map)
        feature_map['{}:False'.format(feature)] = len(feature_map)
    for feature in features['qualitative']:
        for val in feature_summary[feature]:
            feature_map['{}:{}'.format(feature, val)] = len(feature_map)
    for val in feature_summary[features['target']]:
        label_map[val] = len(label_map)
    
    feature_matrix = dok_matrix((len(alldata), len(feature_map)))
    label_matrix = dok_matrix((len(alldata), len(label_map)))
    for i, datum in enumerate(alldata):
        for feature in features['quantitative']:
            val = datum.get(feature)
            z_val = (val - feature_summary[feature]['mean'])/feature_summary[feature]['std'] if val is not None else 0.
            feature_matrix[i, feature_map[feature]] = z_val
        for feature in features['binary']:
            val = datum.get(feature)
            if val is not None:
                if val:
                    feature_matrix[i, feature_map['{}:True'.format(feature)]] = 1.
                else:
                    feature_matrix[i, feature_map['{}:False'.format(feature)]] = 1.
        for feature in features['qualitative']:
            val = datum.get(feature)
            if val is not None and val in feature_summary[feature].keys():
                feature_matrix[i, feature_map['{}:{}'.format(feature, val)]] = 1.
        
        label = datum.get(features['target'])
        if label is not None and label in feature_summary[features['target']]:
            label_matrix[i, label_map[label]] = 1.
                
    return feature_matrix, label_matrix, feature_map, label_map

In [10]:
X, Y, fmap, lmap = transform_data_to_featurevector(alldata, features, summary)

In [11]:
X

<10000x9 sparse matrix of type '<class 'numpy.float64'>'
	with 50000 stored elements in Dictionary Of Keys format>

In [12]:
Y

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Dictionary Of Keys format>

# TN Model

In [58]:
class QuantumTNLayer(tf.keras.layers.Layer):
    def __init__(self, nboutputs, nearzero_std=1e-4):
        super(QuantumTNLayer, self).__init__()
        self.nboutputs = nboutputs
        self.nearzero_std = nearzero_std
        
    def build(self, input_shape):
        self.nbdata = input_shape[0]
        self.vecdim = input_shape[1]
        
        self.tr_var = tf.Variable(tf.random.normal((self.nboutputs, self.vecdim),
                                                   mean=0.0,
                                                   stddev=self.nearzero_std) + \
                                  tf.eye(self.nboutputs, num_columns=self.vecdim),
                                  name='tr_var',
                                  trainable=True)        

        self.tr_node = tn.Node(self.tr_var,
                               backend='tensorflow',
                               name='tr_node')
        
    @tf.function    
    def infer_single_datum(self, input):
        input_node = tn.Node(input, backend='tensorflow')
        edge = input_node[0] ^ self.tr_node[1]
        final_node = tn.contract(edge)
        print(final_node.tensor)
        return final_node.tensor
        
    def call(self, X):
        Ypred = tf.vectorized_map(self.infer_single_datum, X)
#         Ypred = tf.reshape(Ypred, (-1, self.nboutputs))
        return Ypred

In [59]:
tn_model = tf.keras.Sequential([
    tf.keras.Input(shape=(len(fmap),)),
    QuantumTNLayer(len(lmap)),
#     tf.keras.layers.Dense(len(lmap)),
#     tf.keras.layers.Softmax()
])
tn_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), 
                 loss=tf.keras.losses.CategoricalCrossentropy())

Tensor("Tensordot:0", shape=(3,), dtype=float32)


In [52]:
tn_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantum_tn_layer_6 (QuantumT (None, 3)                 27        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [53]:
tn_model.trainable_variables[0].numpy()

array([[ 9.99826372e-01, -1.33002231e-05,  1.63424484e-04,
         1.25248931e-04, -2.01863062e-04,  9.11986062e-05,
        -3.95352181e-05, -9.97562238e-05,  1.10099434e-04],
       [ 1.32149711e-04,  9.99988377e-01, -1.84878736e-05,
        -4.84912416e-05, -3.97471995e-05, -6.96541538e-05,
        -2.16010929e-04,  9.22362815e-05, -1.76564299e-05],
       [-1.79152121e-04,  1.51232118e-04,  9.99919832e-01,
         8.55462495e-05,  3.36445191e-05,  1.69093444e-04,
         5.86483293e-05, -1.84299352e-04,  2.56294370e-05]], dtype=float32)

In [54]:
X[0:10, :].toarray().shape

(10, 9)

In [55]:
np.matmul(tn_model.trainable_variables[0].numpy(), X[0:10, :].T.toarray())

array([[ 1.32879070e+00,  5.01774340e-01,  6.64570282e-01,
         3.94539627e-01, -1.03287273e+00, -9.95835938e-01,
        -7.48312507e-02, -1.10184323e+00, -1.05381826e+00,
        -1.67577136e-01],
       [ 9.62675461e-01,  9.44974174e-01,  1.50395482e+00,
         9.10115986e-01, -9.31238699e-01, -1.08433218e+00,
         1.08017755e+00, -1.26552132e+00, -1.20097176e+00,
         1.17071140e+00],
       [ 8.53275685e-05, -1.21505172e-05,  2.53194417e-04,
         1.79463308e-06,  1.14626673e-04,  8.48370058e-05,
         1.11626071e-04,  2.86415364e-04,  7.75880476e-05,
         5.20449055e-04]])

In [56]:
tn_model.predict(X[0:10, :].toarray())

Tensor("sequential_6/quantum_tn_layer_6/loop_body/Tensordot:0", shape=(3,), dtype=float32)


array([[ 1.32879066e+00,  9.62675393e-01,  8.53275706e-05],
       [ 5.01774371e-01,  9.44974124e-01, -1.21505109e-05],
       [ 6.64570272e-01,  1.50395477e+00,  2.53194419e-04],
       [ 3.94539624e-01,  9.10115957e-01,  1.79464041e-06],
       [-1.03287268e+00, -9.31238770e-01,  1.14626680e-04],
       [-9.95835900e-01, -1.08433211e+00,  8.48370037e-05],
       [-7.48312548e-02,  1.08017766e+00,  1.11626076e-04],
       [-1.10184312e+00, -1.26552129e+00,  2.86415365e-04],
       [-1.05381835e+00, -1.20097172e+00,  7.75880690e-05],
       [-1.67577133e-01,  1.17071140e+00,  5.20449015e-04]], dtype=float32)

In [25]:
X[:10, :]

<10x9 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Dictionary Of Keys format>

In [57]:
tn_model.fit(X.toarray(), Y.toarray(), epochs=100, batch_size=100)

Train on 10000 samples
Epoch 1/100
Tensor("sequential_6/quantum_tn_layer_6/loop_body/Tensordot:0", shape=(3,), dtype=float32)
  100/10000 [..............................] - ETA: 4s

ValueError: No gradients provided for any variable: ['quantum_tn_layer_6/tr_var:0'].